## LGD Model

In [34]:
#import Libraries
import warnings
warnings.filterwarnings("ignore")
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [35]:
data = pd.read_csv('loan_data/lgd_preprocessed_data.csv', index_col=0)
data1 = pd.read_csv('loan_data/lgd_preprocessed_data1.csv', index_col=0)

In [36]:
pd.options.display.max_columns=None

In [37]:
data.head()

,id,member_id,loan_amnt,funded_amnt,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,initial_list_status,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq,total_rev_hi_lim,emp_length_int,term_int,earliest_cr_line_mnths,mnths_since_issue_date,grade_freq,sub_grade_freq,home_ownership_freq,purpose_freq,addr_state_freq,verification_status:Not Verified,verification_status:Source Verified,verification_status:Verified,initial_list_status:f,initial_list_status:w,recovery_rate,CCF,recovery_rate_0_1
1,1077430,1314167,2500,2500,15.27,59.83,C,C4,RENT,30000.0,Source Verified,Charged Off,car,309xx,GA,1.00,0.0,5.0,0.0,0.0,3.0,0.0,4.0,f,456.46,435.17,117.08,0.0,2500.0,0,60,224.0,72.0,12069,2441,20258,438,1313,0,1,0,1,0,0.046832,0.817416,1
8,1071795,1306957,5600,5600,21.28,152.39,F,F2,OWN,40000.0,Source Verified,Charged Off,small_business,958xx,CA,5.55,0.0,2.0,0.0,0.0,11.0,0.0,13.0,f,162.02,294.94,189.06,0.0,5600.0,4,60,164.0,72.0,2778,600,3785,1394,7011,0,1,0,1,0,0.033761,0.971068,1
9,1071570,1306721,5375,5375,12.69,121.45,B,B5,RENT,15000.0,Verified,Charged Off,other,774xx,TX,18.08,0.0,0.0,0.0,0.0,2.0,0.0,3.0,f,673.48,533.42,269.29,0.0,5375.0,0,60,159.0,72.0,9242,2045,20258,2892,2883,0,0,1,1,0,0.050100,0.874701,1
12,1064687,1298717,9000,9000,13.49,305.38,C,C1,RENT,30000.0,Source Verified,Charged Off,debt_consolidation,245xx,VA,10.08,0.0,1.0,0.0,0.0,4.0,0.0,9.0,f,1256.14,570.26,444.30,0.0,9000.0,0,36,164.0,72.0,12069,2363,20258,26127,1348,0,1,0,1,0,0.049367,0.860429,1
14,1069057,1303503,10000,10000,10.65,325.74,B,B2,RENT,100000.0,Source Verified,Charged Off,other,951xx,CA,7.06,0.0,2.0,0.0,0.0,14.0,0.0,29.0,f,5433.47,1393.42,645.10,0.0,10000.0,3,36,319.0,72.0,9242,1554,20258,2892,7011,0,1,0,1,0,0.064510,0.456653,1


In [38]:
data.shape, data1.shape

((43236, 46), (43236, 162))

#### Splitting the data

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
#data 1
X_train, X_test, y_train, y_test = train_test_split(data.drop(['recovery_rate', 'recovery_rate_0_1', 'CCF'], axis=1),
                                                   data['recovery_rate_0_1'], random_state=42, stratify=data['recovery_rate_0_1'])

In [41]:
#data 2
X_train1, X_test1, y_train1, y_test1 = train_test_split(data1.drop(['recovery_rate', 'recovery_rate_0_1', 'CCF'], axis=1),
                                                   data1['recovery_rate_0_1'], random_state=42, stratify=data1['recovery_rate_0_1'])

In [42]:
data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'int_rate',
       'installment', 'grade', 'sub_grade', 'home_ownership', 'annual_inc',
       'verification_status', 'loan_status', 'purpose', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'total_acc', 'initial_list_status', 'total_rec_prncp',
       'total_rec_int', 'recoveries', 'acc_now_delinq', 'total_rev_hi_lim',
       'emp_length_int', 'term_int', 'earliest_cr_line_mnths',
       'mnths_since_issue_date', 'grade_freq', 'sub_grade_freq',
       'home_ownership_freq', 'purpose_freq', 'addr_state_freq',
       'verification_status:Not Verified',
       'verification_status:Source Verified', 'verification_status:Verified',
       'initial_list_status:f', 'initial_list_status:w', 'recovery_rate',
       'CCF', 'recovery_rate_0_1'],
      dtype='object')

In [43]:
data_features_all = [
'funded_amnt', 'int_rate', 'installment', 'annual_inc',
'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
'pub_rec', 'total_acc', 'acc_now_delinq', 'total_rev_hi_lim',
'emp_length_int', 'term_int', 'earliest_cr_line_mnths',
'mnths_since_issue_date', 'grade_freq', 'sub_grade_freq',
'home_ownership_freq', 'purpose_freq', 'addr_state_freq', 'initial_list_status:w', 'initial_list_status:f',
'verification_status:Not Verified', 'verification_status:Source Verified', 'verification_status:Verified'
]

data_ref_cat = [
'verification_status:Verified',
'initial_list_status:f'
]

In [44]:
data1.columns.values

array(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'int_rate',
       'installment', 'grade', 'sub_grade', 'home_ownership',
       'annual_inc', 'verification_status', 'loan_status', 'purpose',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'total_acc', 'initial_list_status', 'total_rec_prncp',
       'total_rec_int', 'recoveries', 'acc_now_delinq',
       'total_rev_hi_lim', 'emp_length_int', 'term_int',
       'earliest_cr_line_mnths', 'mnths_since_issue_date', 'grade:A',
       'grade:B', 'grade:C', 'grade:D', 'grade:E', 'grade:F', 'grade:G',
       'sub_grade:A1', 'sub_grade:A2', 'sub_grade:A3', 'sub_grade:A4',
       'sub_grade:A5', 'sub_grade:B1', 'sub_grade:B2', 'sub_grade:B3',
       'sub_grade:B4', 'sub_grade:B5', 'sub_grade:C1', 'sub_grade:C2',
       'sub_grade:C3', 'sub_grade:C4', 'sub_grade:C5', 'sub_grade:D1',
       'sub_grade:D2', 'sub_grade:D3', 'sub_g

In [78]:
data1_features_all = [
'funded_amnt', 'int_rate', 'installment', 'annual_inc',
'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
'pub_rec', 'total_acc', 'acc_now_delinq', 'total_rev_hi_lim',
'emp_length_int', 'term_int', 'earliest_cr_line_mnths',
'mnths_since_issue_date', 'initial_list_status:f', 'initial_list_status:w',
'grade:A', 'grade:B', 'grade:C', 'grade:D', 'grade:E', 'grade:F', 'grade:G',
'home_ownership:MORTGAGE',
'home_ownership:NONE', 'home_ownership:OTHER', 'home_ownership:OWN', 'home_ownership:RENT',
'verification_status:Not Verified', 'verification_status:Source Verified',
'verification_status:Verified',
'purpose:car', 'purpose:credit_card', 'purpose:debt_consolidation',
'purpose:educational', 'purpose:home_improvement', 'purpose:house',
'purpose:major_purchase', 'purpose:medical', 'purpose:moving',
'purpose:other', 'purpose:renewable_energy',
'purpose:small_business', 'purpose:vacation', 'purpose:wedding', 'total_rev_hi_lim'
]

data1_ref_cat = [
'grade:G', 'home_ownership:RENT', 'purpose:credit_card', 'initial_list_status:f'
]

In [ ]:
X_train = X_train[data_features_all]
X_train1 = X_train1[data1_features_all]

In [77]:
X_train1.corr()

,funded_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,acc_now_delinq,total_rev_hi_lim,emp_length_int,term_int,earliest_cr_line_mnths,mnths_since_issue_date,initial_list_status:w,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:ANY,home_ownership:MORTGAGE,home_ownership:NONE,home_ownership:OTHER,home_ownership:OWN,verification_status:Not Verified,verification_status:Source Verified,verification_status:Verified,purpose:car,purpose:debt_consolidation,purpose:educational,purpose:home_improvement,purpose:house,purpose:major_purchase,purpose:medical,purpose:moving,purpose:other,purpose:renewable_energy,purpose:small_business,purpose:vacation,purpose:wedding,total_rev_hi_lim
funded_amnt,1.000000,0.273175,0.951217,0.466398,0.050126,0.026586,-0.019580,0.001760,-0.070632,0.214219,-0.052266,0.269018,0.023346,0.387513,0.163526,0.448341,0.158793,-0.145154,0.051545,-0.081559,-0.121946,-0.066924,0.001101,0.151132,0.137874,NaN,0.209837,-0.004486,-0.007085,-0.029639,-0.353018,0.034756,0.288987,-0.069861,0.124625,-0.040199,0.011907,0.000067,-0.043770,-0.075054,-0.080178,-0.144656,-0.012484,0.034202,-0.071523,-0.032983,0.387513
int_rate,0.273175,1.000000,0.257870,0.059237,0.125967,0.076526,0.143775,0.052200,0.020297,0.061383,0.035301,0.019439,0.032303,0.011930,0.046703,0.449678,-0.082652,-0.216300,0.041896,-0.474369,-0.504730,-0.152966,0.228439,0.423482,0.446144,NaN,-0.029150,-0.002810,-0.007118,0.013677,-0.248971,0.039947,0.189577,-0.043303,0.065766,-0.028386,-0.021831,0.018081,-0.030110,0.018236,0.022780,0.064767,-0.003463,0.035551,-0.011891,-0.008096,0.011930
installment,0.951217,0.257870,1.000000,0.467361,0.048221,0.041114,0.000390,0.011080,-0.065719,0.213068,-0.041036,0.249809,0.029549,0.365418,0.144765,0.200080,0.139244,-0.147689,0.040845,-0.077998,-0.102675,-0.071081,0.009498,0.126609,0.128975,NaN,0.174467,-0.002186,-0.004279,-0.025700,-0.302483,0.013606,0.262535,-0.071817,0.114892,-0.037911,0.005737,0.000025,-0.044913,-0.071662,-0.074050,-0.137087,-0.008796,0.046419,-0.069534,-0.028205,0.365418
annual_inc,0.466398,0.059237,0.467361,1.000000,-0.182227,0.087365,0.091233,0.038315,-0.031315,0.202190,0.019764,0.275950,0.046863,0.293361,0.124699,0.125856,0.183728,-0.023043,0.008827,0.007347,-0.033074,-0.034701,-0.018294,0.047551,0.041412,NaN,0.218909,-0.002601,-0.007324,-0.049224,-0.136424,0.060037,0.068696,-0.017119,-0.013862,-0.017267,0.065083,0.000186,0.000481,0.002566,-0.023549,-0.023571,-0.001169,0.063585,-0.019140,-0.000043,0.293361
dti,0.050126,0.125967,0.048221,-0.182227,1.000000,-0.010456,-0.053076,-0.015042,-0.016524,0.316500,-0.026211,0.258311,0.006484,0.140145,0.044711,0.055985,0.039216,-0.230416,0.091485,-0.073494,-0.058341,0.018873,0.025520,0.048590,0.016707,NaN,0.010035,0.000726,-0.006255,0.005509,-0.124066,-0.029588,0.140117,-0.055764,0.130569,-0.039230,-0.074337,-0.031040,-0.059747,-0.027696,-0.040080,-0.058953,-0.009641,-0.101559,-0.011690,-0.030040,0.140145
delinq_2yrs,0.026586,0.076526,0.041114,0.087365,-0.010456,1.000000,0.015663,-0.047530,-0.009845,0.055175,0.001931,0.147618,0.105096,-0.010769,0.012693,-0.004576,0.091066,-0.080363,0.034212,-0.046938,-0.041628,-0.006794,0.023259,0.028550,0.029087,NaN,0.058950,0.021215,0.000795,0.004052,-0.025211,0.027988,-0.002885,-0.004830,0.010890,-0.003466,0.022221,-0.000463,0.007350,0.007453,-0.004927,-0.002325,0.004011,0.009853,-0.005484,0.003643,-0.010769
inq_last_6mths,-0.019580,0.143775,0.000390,0.091233,-0.053076,0.015663,1.000000,0.021766,0.024042,0.081956,0.028547,0.105325,-0.003355,0.003112,-0.014706,-0.002149,0.027350,0.173274,-0.035872,-0.077274,-0.121389,-0.015116,0.052558,0.065875,0.079466,NaN,0.065835,0.001272,0.021078,0.004371,0.005614,-0.025538,0.018448,0.006169,-0.048254,0.060416,0.064214,0.030978,0.020237,0.007836,0.018549,0.025921,0.004367,0.047045,-0.000473,0.019999,0.003112
mths_since_last_delinq,0.001760,0.052200,0.011080,0.038315,-0.015042,-0.047530,0.02176

In [57]:
X_test = X_test[data_features_all]
X_test1 = X_test1[data1_features_all]

In [49]:
X_train = X_train.drop(data_ref_cat, axis=1)
X_train1 = X_train1.drop(data1_ref_cat, axis=1)

In [81]:
X_train1 = X_train1.drop('home_ownership:ANY', axis=1)
X_test1 = X_test1.drop('home_ownership:ANY', axis=1)

In [58]:
X_test = X_test.drop(data_ref_cat, axis=1)
X_test1 = X_test1.drop(data1_ref_cat, axis=1)

### Logistic Regression Model

In [50]:
from sklearn.linear_model import LogisticRegression

In [53]:
import scipy.stats as stat

class LogisticRegression_with_p_values:
    def __init__(self, *args, **kwargs):
        self.model = LogisticRegression(*args, **kwargs)
        
    def fit(self, X, y):
        self.model.fit(X,y)
        denom = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom, (X.shape[1], 1)).T
        F_ij = np.dot((X / denom).T, X)
        cramer_Rao = np.linalg.inv(F_ij)
        sigma_estimates = np.sqrt(np.diagonal(cramer_Rao))
        z_scores = self.model.coef_[0] / sigma_estimates
        p_values = [stat.norm.sf(abs(x)) * 2 for x in z_scores]
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        self.p_values = p_values

#### Model 1

In [55]:
reg = LogisticRegression_with_p_values(random_state=42)

In [56]:
reg.fit(X_train, y_train)

In [60]:
pred = reg.model.predict_proba(X_test)

In [62]:
pred = pred[:][:, 1]

In [63]:
pred

array([0.48981632, 0.68618238, 0.4760504 , ..., 0.68411397, 0.54188733,
       0.53191048])

In [65]:
feature_name = X_train.columns.values

In [72]:
p_values = reg.p_values
p_values = np.append(np.nan, np.array(p_values))

In [73]:
summary_table = pd.DataFrame(columns=['Feature name'], data=feature_name)
summary_table['Coefficients'] = np.transpose(reg.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table['p_values'] = p_values

In [74]:
summary_table

,Feature name,Coefficients,p_values
0,Intercept,2.011501e-05,NaN
1,funded_amnt,-2.352872e-06,8.012220e-01
2,int_rate,7.837136e-04,8.433740e-01
3,installment,3.853779e-04,1.849792e-01
4,annual_inc,5.186476e-07,1.521129e-01
5,dti,-1.288513e-03,4.354897e-01
6,delinq_2yrs,-1.775861e-05,9.990675e-01
7,inq_last_6mths,1.409982e-04,9.880874e-01
8,mths_since_last_delinq,-2.546602e-04,6.264189e-01
9,mths_since_last_record,-4.772714e-03,5.350872e-14


#### Model 2

In [75]:
reg1 = LogisticRegression_with_p_values(random_state=42)

In [82]:
reg1.fit(X_train1, y_train1)

In [83]:
pred1 = reg1.model.predict_proba(X_test1)

In [85]:
pred1 = pred1[:][:, 1]

In [86]:
feature_name1 = X_train1.columns.values

In [ ]:
p_values = reg.p_values
p_values = np.append(np.nan, np.array(p_values))

In [ ]:
summary_table = pd.DataFrame(columns=['Feature name'], data=feature_name)
summary_table['Coefficients'] = np.transpose(reg.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table['p_values'] = p_values

In [ ]:
summary_table